In [1]:
# !apt update
# !apt install ffmpeg libsm6 libxext6 -y

In [2]:
# !pip install opencv-python
# !pip install --upgrade pip

In [3]:
import numpy as np
import os
import cv2
import time
import math
import json
from pathlib import Path
from PIL import Image, ImageColor
import tensorflow as tf

from tensorflow import keras
from scipy import io
import matplotlib.pyplot as plt
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from types import MethodType
import random
import six
import json
from tqdm import tqdm
import itertools

from collections import OrderedDict

In [4]:
IMG_ORD = "channels_last"
MERGE_AXIS = -1

pretrained_url ='../input/dataset-map/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
#     pretrained_url = open('../input/dataset-map/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'r')

classes_file = np.transpose(np.array(json.load(open('../input/dataset-map/dataset_map/dataset_map/classes.json'))))[2]
print(classes_file)
    
    
class_colors = [ImageColor.getcolor('#' + hexc, "RGB") for hexc in classes_file]
print(class_colors)

In [5]:
def get_colored_segmentation_image( seg_arr  , n_classes , colors=class_colors ):
    output_height = seg_arr.shape[0]
    output_width = seg_arr.shape[1]

    seg_img = np.zeros((output_height, output_width, 3))

    for c in range(n_classes):
        seg_img[:, :, 0] += ((seg_arr[:, :] == c)*(colors[c][0])).astype('uint8')
        seg_img[:, :, 1] += ((seg_arr[:, :] == c)*(colors[c][1])).astype('uint8')
        seg_img[:, :, 2] += ((seg_arr[:, :] == c)*(colors[c][2])).astype('uint8')

    return seg_img 

In [6]:
def visualize_segmentation( seg_arr , inp_img=None  , n_classes=None , 
    colors=class_colors , class_names=None , overlay_img=False , show_legends=False , 
    prediction_width=None , prediction_height=None  ):
    

    if n_classes is None:
        n_classes = np.max(seg_arr)

    seg_img = get_colored_segmentation_image( seg_arr  , n_classes , colors=colors )

    if not inp_img is None:
        orininal_h = inp_img.shape[0]
        orininal_w = inp_img.shape[1]
        seg_img = cv2.resize(seg_img, (orininal_w, orininal_h))


    if (not prediction_height is None) and  (not prediction_width is None):
        seg_img = cv2.resize(seg_img, (prediction_width, prediction_height ))
        if not inp_img is None:
            inp_img = cv2.resize(inp_img, (prediction_width, prediction_height ))


    if overlay_img:
        assert not inp_img is None
        seg_img = overlay_seg_image( inp_img , seg_img  )


    if show_legends:
        assert not class_names is None
        legend_img = get_legends(class_names , colors=colors )

        seg_img = concat_lenends( seg_img , legend_img )


    return seg_img

In [7]:
def check_input(image_input):
    if type(image_input) is np.ndarray:
        img = image_input
    elif  isinstance(image_input, six.string_types)  :
        img = cv2.imread(image_input, 1)
    else:
        raise DataLoaderError("get_segmentation_array: Can't process input type {0}".format(str(type(image_input))))
    
    return img

In [8]:
def get_image_array(image_input, width, height, imgNorm="sub_mean"):
    
    img = check_input(image_input)

    if imgNorm == "sub_and_divide":
        img = np.float32(cv2.resize(img, (width, height))) / 127.5 - 1
    elif imgNorm == "sub_mean":
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img[:, :, 0] -= 103.939
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 123.68
        img = img[:, :, ::-1]
    elif imgNorm == "divide":
        img = cv2.resize(img, (width, height))
        img = img.astype(np.float32)
        img = img/255.0
        
    return img

def get_segmentation_array(image_input, nClasses, width, height, no_reshape=False):

    seg_labels = np.zeros((height, width, nClasses))

    img = check_input(image_input)

    img = cv2.resize(img, (width, height), interpolation=cv2.INTER_NEAREST)
    img = img[:, :, 0]

    for c in range(nClasses):
        seg_labels[:, :, c] = (img == c).astype(int)

    if not no_reshape:
        seg_labels = np.reshape(seg_labels, (width*height, nClasses))

    return seg_labels

In [9]:
def image_segmentation_generator(images_path, segs_path, batch_size,
                                 n_classes, input_height, input_width,
                                 output_height, output_width):

    img_seg_pairs = get_pairs_from_paths(images_path, segs_path)
    random.shuffle(img_seg_pairs)
    zipped = itertools.cycle(img_seg_pairs)

    while True:
        X = []
        Y = []
        for _ in range(batch_size):
            im, seg = next(zipped)

            im = cv2.imread(im, 1)
            seg = cv2.imread(seg, 1)

            X.append(get_image_array(im, input_width, input_height))
            Y.append(get_segmentation_array(seg, n_classes, output_width, output_height))

        yield np.array(X), np.array(Y)

In [10]:
def get_pairs_from_paths(images_path, segs_path, ignore_non_matching=False):
    
    image_files = []
    segmentation_files = {}

    for dir_entry in os.listdir(images_path):
        if os.path.isfile(os.path.join(images_path, dir_entry)):
            file_name, file_extension = os.path.splitext(dir_entry)
            
            image_files.append((file_name, file_extension, os.path.join(images_path, dir_entry)))

    for dir_entry in os.listdir(segs_path):
        if os.path.isfile(os.path.join(segs_path, dir_entry)):
            file_name, file_extension = os.path.splitext(dir_entry)
            
            segmentation_files[file_name] = (file_extension, os.path.join(segs_path, dir_entry))

    return_value = []
    
    for image_file, _, image_full_path in image_files:
        if image_file in segmentation_files:
            return_value.append((image_full_path, segmentation_files[image_file][1]))
        elif ignore_non_matching:
            continue

    return return_value

In [11]:
def verify_segmentation_dataset(images_path, segs_path, n_classes):
    try:
        img_seg_pairs = get_pairs_from_paths(images_path, segs_path)

        return_value = True
        for im_fn, seg_fn in tqdm(img_seg_pairs):
            img = cv2.imread(im_fn)
            seg = cv2.imread(seg_fn)
            
            if not img.shape == seg.shape:
                return_value = False
            else:
                max_pixel_value = np.max(seg[:, :, 0])
                if max_pixel_value >= n_classes:
                    return_value = False
                    
        if return_value:
            print("Dataset verified!")
        else:
            print("Dataset not verified!")
        return return_value
    except Exception as e:
        print(str(e))
        return False

In [12]:
def evaluate(model=None ,inp_images=None ,annotations=None, inp_images_dir=None, annotations_dir=None, checkpoints_path=None):
        
    if inp_images is None:
        paths = get_pairs_from_paths(inp_images_dir , annotations_dir)
        paths = list(zip(*paths))
        inp_images = list(paths[0])
        annotations = list(paths[1])
        
    tp = np.zeros(model.n_classes)
    fp = np.zeros(model.n_classes)
    fn = np.zeros(model.n_classes)
    n_pixels = np.zeros(model.n_classes)
    
    for inp, ann in tqdm(zip(inp_images, annotations)):
        pr = predict(model , inp )
        gt = get_segmentation_array(ann, model.n_classes, model.output_width, model.output_height, no_reshape=True)
        gt = gt.argmax(-1)
        pr = pr.flatten()
        gt = gt.flatten()
                
        for cl_i in range(model.n_classes):
            tp[ cl_i ] += np.sum( (pr == cl_i) * (gt == cl_i) )
            fp[ cl_i ] += np.sum( (pr == cl_i) * ((gt != cl_i)) )
            fn[ cl_i ] += np.sum( (pr != cl_i) * ((gt == cl_i)) )
            n_pixels[ cl_i ] += np.sum( gt == cl_i  )
            
    cl_wise_score = tp / (tp + fp + fn + 0.000000000001)
    n_pixels_norm = n_pixels /  np.sum(n_pixels)
    frequency_weighted_IU = np.sum(cl_wise_score * n_pixels_norm)
    mean_IU = np.mean(cl_wise_score)
    
    return {"frequency_weighted_IU":frequency_weighted_IU , "mean_IU":mean_IU , "class_wise_IU":cl_wise_score }

In [13]:
def predict(model=None, inp=None, out_fname=None, checkpoints_path=None,overlay_img=False ,
    class_names=None, show_legends=False, colors=class_colors, prediction_width=None, prediction_height=None):
    
    inp = cv2.imread(inp)
    
    orininal_h, orininal_w, _ = inp.shape

    output_width = model.output_width
    output_height = model.output_height
    input_width = model.input_width
    input_height = model.input_height
    n_classes = model.n_classes

    x = get_image_array(inp, input_width, input_height)
    pr = model.predict(np.array([x]))[0]
    pr = pr.reshape((output_height, output_width, n_classes)).argmax(axis=2)

    seg_img = visualize_segmentation(pr, inp, n_classes=n_classes, colors=colors, overlay_img=overlay_img,
                                     show_legends=show_legends, class_names=class_names,
                                     prediction_width=prediction_width, prediction_height=prediction_height)

    if out_fname is not None:
        cv2.imwrite(out_fname, seg_img)

    return pr

In [14]:
def train(model,
          train_images,
          train_annotations,
          input_height=None,
          input_width=None,
          n_classes=None,
          verify_dataset=True,
          checkpoints_path=None,
          epochs=5,
          batch_size=2,
          validate=False,
          val_images=None,
          val_annotations=None,
          val_batch_size=2,
          auto_resume_checkpoint=False,
          load_weights=None,
          steps_per_epoch=200,
          val_steps_per_epoch=100,
          gen_use_multiprocessing=False,
          ignore_zero_class=False , 
          optimizer_name='adadelta' , do_augment=False , augmentation_name="aug_all"
          ):

    
    
    
    if isinstance(model, six.string_types):
        if (input_height is not None) and (input_width is not None):
            model = model_from_name[model](
                n_classes, input_height=input_height, input_width=input_width)
        else:
            model = model_from_name[model](n_classes)

    n_classes = model.n_classes
    input_height = model.input_height
    input_width = model.input_width
    output_height = model.output_height
    output_width = model.output_width

    if optimizer_name is not None:

        if ignore_zero_class:
            loss_k = masked_categorical_crossentropy
        else:
            loss_k = 'categorical_crossentropy'

        model.compile(loss= loss_k, optimizer=optimizer_name, metrics=['accuracy'])

    if load_weights is not None and len(load_weights) > 0:
        print("Loading weights from ", load_weights)
        model.load_weights(load_weights)

    if verify_dataset:
        print("Verifying training dataset")
        verified = verify_segmentation_dataset(train_images, train_annotations, n_classes)
        assert verified

    train_gen = image_segmentation_generator(train_images, train_annotations, batch_size, n_classes,
                                             input_height, input_width, output_height, output_width)

    for ep in range(epochs):
        print("Starting Epoch ", ep)
        model.fit_generator(train_gen, steps_per_epoch, epochs=1, use_multiprocessing=True)
        print("Finished Epoch", ep)

In [15]:
def get_segmentation_model(input, output):

    img_input = input
    o = output

    o_shape = Model(img_input, o).output_shape
    i_shape = Model(img_input, o).input_shape

    output_height = o_shape[1]
    output_width = o_shape[2]
    input_height = i_shape[1]
    input_width = i_shape[2]
    n_classes = o_shape[3]
    o = (Reshape((output_height*output_width, -1)))(o)

    o = (Activation('softmax'))(o)
#     model = tf.keras.Sequential(img_input, o)
#     model = tf.keras.Sequential()
    model = Model(img_input, o)
    model.output_width = output_width
    model.output_height = output_height
    model.n_classes = n_classes
    model.input_height = input_height
    model.input_width = input_width
    model.model_name = ""

    model.train = MethodType(train, model)
    model.predict_segmentation = MethodType(predict, model)
    model.evaluate_segmentation = MethodType(evaluate, model)

    return model

In [16]:
def get_vgg_encoder(input_height,  input_width, pretrained='imagenet'):

    assert input_height % 32 == 0
    assert input_width % 32 == 0

    img_input = Input(shape=(input_height, input_width, 3))
    
# Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same',
               name='block1_conv1', data_format=IMG_ORD)(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same',
               name='block1_conv2', data_format=IMG_ORD)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool',
                     data_format=IMG_ORD)(x)
    f1 = x
# Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same',
               name='block2_conv1', data_format=IMG_ORD)(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same',
               name='block2_conv2', data_format=IMG_ORD)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool',
                     data_format=IMG_ORD)(x)
    f2 = x

# Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='block3_conv1', data_format=IMG_ORD)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='block3_conv2', data_format=IMG_ORD)(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same',
               name='block3_conv3', data_format=IMG_ORD)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool',
                     data_format=IMG_ORD)(x)
    f3 = x

# Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block4_conv1', data_format=IMG_ORD)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block4_conv2', data_format=IMG_ORD)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block4_conv3', data_format=IMG_ORD)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool',
                     data_format=IMG_ORD)(x)
    f4 = x

# Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block5_conv1', data_format=IMG_ORD)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block5_conv2', data_format=IMG_ORD)(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same',
               name='block5_conv3', data_format=IMG_ORD)(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool',
                     data_format=IMG_ORD)(x)
    f5 = x

    if pretrained == 'imagenet':
        VGG_Weights_path = pretrained_url
        
        Model(img_input, x).load_weights(VGG_Weights_path)

    return img_input, [f1, f2, f3, f4, f5]

In [17]:
def _unet(n_classes, encoder, l1_skip_conn=True, input_height=200, input_width=200):

    img_input, [f1, f2, f3, f4, _] = encoder(input_height=input_height, input_width=input_width)

#     o = f4

#     o = (ZeroPadding2D((1, 1), data_format=IMG_ORD))(o)
#     o = (Conv2D(512, (3, 3), padding='valid', data_format=IMG_ORD))(o)
#     o = (BatchNormalization())(o)

#     o = (UpSampling2D((2, 2), data_format=IMG_ORD))(o)
    
#     o = (concatenate([o, f3], axis=MERGE_AXIS))
    o = f3
    o = (ZeroPadding2D((1, 1), data_format=IMG_ORD))(o)
    o = (Conv2D(256, (3, 3), padding='valid', data_format=IMG_ORD))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=IMG_ORD))(o)
    o = (concatenate([o, f2], axis=MERGE_AXIS))
    o = (ZeroPadding2D((1, 1), data_format=IMG_ORD))(o)
    o = (Conv2D(128, (3, 3), padding='valid', data_format=IMG_ORD))(o)
    o = (BatchNormalization())(o)

    o = (UpSampling2D((2, 2), data_format=IMG_ORD))(o)

    if l1_skip_conn:
        o = (concatenate([o, f1], axis=MERGE_AXIS))

    o = (ZeroPadding2D((1, 1), data_format=IMG_ORD))(o)
    o = (Conv2D(64, (3, 3), padding='valid', data_format=IMG_ORD))(o)
    o = (BatchNormalization())(o)

    o = Conv2D(n_classes, (3, 3), padding='same',data_format=IMG_ORD)(o)

    model = get_segmentation_model(img_input, o)

    return model

In [18]:
# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)

In [19]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)



In [20]:
# BATCH_SIZE = 16 * tpu_strategy.num_replicas_in_sync
# print(BATCH_SIZE)

In [21]:
def vgg_unet(n_classes, input_height, input_width, encoder_level=3):

    model = _unet(n_classes, get_vgg_encoder,input_height=input_height, input_width=input_width)
    model.model_name = "vgg_unet"
    return model

n_classes = len(class_colors)

model = vgg_unet(n_classes=n_classes,  input_height=608, input_width=800)
model_from_name = {}
model_from_name["vgg_unet"] = vgg_unet

In [22]:
epochs = 1

In [23]:
model.train( 
    train_images =  "../input/dataset-map/dataset_map/dataset_map/rgb",
    train_annotations = "../input/dataset-map/dataset_map/dataset_map/label",
    epochs=epochs#, batch_size=BATCH_SIZE
)

In [24]:
start = time.time()

input_image = "../input/dataset-map/dataset_map/dataset_map/rgb/IMG_0197.tif"
out = model.predict_segmentation(
    inp=input_image,
    out_fname="out.png"
)

fig, axs = plt.subplots(1, 3, figsize=(20, 20), constrained_layout=True)


img_orig = Image.open(input_image)
axs[0].imshow(img_orig)
axs[0].set_title('original image-002.jpg')
axs[0].grid(False)

axs[1].imshow(out)
axs[1].set_title('prediction image-out.png')
axs[1].grid(False)

validation_image = "../input/dataset-map/dataset_map/dataset_map/label/IMG_0197.tif"
axs[2].imshow( Image.open(validation_image))
axs[2].set_title('true label image-002.png')
axs[2].grid(False)


done = time.time()
elapsed = done - start

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d0ebdbe2-6239-480a-91f2-f925ec1458cf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>